In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from tensorflow.keras.utils  import plot_model
from collections  import Counter
from keras.callbacks import ModelCheckpoint

In [ ]:

df=pd.read_csv('legend.csv') #read into data frame

df.drop(columns="user.id", inplace=True) #drop the userid col, since it is not used
df['emotion'] = df['emotion'].str.lower() #make all the labels lowercase

valid_df = df.sample(frac=0.1) #randomly sample 10% of the data to use as validation data
train_df = df[~df['image'].isin(valid_df['image'])] #delete validation data from training data

test_df = train_df.sample(frac=0.05) #randomly 5% sample remaining data to use as test data, since no other test data exist now
train_df = train_df[~train_df['image'].isin(test_df['image'])] #delete test data from training data

NameError: ignored

# New Section

In [ ]:
categories = train_df['emotion'].unique() #examine how many categories are there
print(categories)
df.groupby('emotion').count() #examine how many samples are in each category

['anger' 'disgust' 'fear' 'neutral' 'happiness' 'surprise' 'sadness'
 'contempt']


,image
emotion,
anger,252
contempt,9
disgust,208
fear,21
happiness,5696
neutral,6868
sadness,268
surprise,368


In [ ]:
import os
os.mkdir('data/train')
os.mkdir('data/valid')
os.mkdir('data/test')
for emotion in categories:
    train_cat = 'data/train/'+emotion
    valid_cat = 'data/valid/'+emotion
    test_cat  = 'data/test/'+emotion
    os.mkdir(train_cat)
    os.mkdir(valid_cat)
    os.mkdir(test_cat)

In [ ]:
import shutil
for index, row in train_df.iterrows():
    file = row['image']
    emotion = row['emotion']

    dest_img_path = 'data/train/' + emotion + '/' + file
    src_img_path = 'images/' + file

    shutil.copy(src_img_path,dest_img_path)

for index, row in valid_df.iterrows():
    file = row['image']
    emotion = row['emotion']

    dest_img_path = 'data/valid/' + emotion + '/' + file
    src_img_path = 'images/' + file

    shutil.copy(src_img_path,dest_img_path)

for index, row in test_df.iterrows():
    file = row['image']
    emotion = row['emotion']

    dest_img_path = 'data/test/' + emotion + '/' + file
    src_img_path = 'images/' + file

    shutil.copy(src_img_path,dest_img_path)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32
img_h, img_w = 224, 224

# this is the augmentation configuration I used for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.01,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='constant')

# this is the augmentation configuration I used for testing:
# only rescaling
valid_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in+

# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train/',  # this is the target directory
        target_size=(img_h, img_w),  # all images will be resized to 224x224
        batch_size=batch_size,
        class_mode='categorical')  #

# this is a similar generator, for validation data
validation_generator = valid_datagen.flow_from_directory(
        'data/valid/',
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
print(train_df.size, valid_df.size)

In [ ]:
from keras.applications.vgg16 import VGG16
#Load the VGG model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in base_model.layers:
    print(layer, layer.trainable)


<keras.engine.input_layer.InputLayer object at 0x7f06cab623d0> False
<keras.layers.convolutional.Conv2D object at 0x7f06ca89fd00> False
<keras.layers.convolutional.Conv2D object at 0x7f06ca8bc790> False
<keras.layers.pooling.MaxPooling2D object at 0x7f06ca8bc4c0> False
<keras.layers.convolutional.Conv2D object at 0x7f06ca044c10> False
<keras.layers.convolutional.Conv2D object at 0x7f06ca044cd0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f06ca044fa0> False
<keras.layers.convolutional.Conv2D object at 0x7f06ca04ff40> False
<keras.layers.convolutional.Conv2D object at 0x7f06ca057cd0> False
<keras.layers.convolutional.Conv2D object at 0x7f06ca04be20> False
<keras.layers.pooling.MaxPooling2D object at 0x7f06ca057ca0> False
<keras.layers.convolutional.Conv2D object at 0x7f06c9feb670> False
<keras.layers.convolutional.Conv2D object at 0x7f06c9feb1c0> False
<keras.layers.convolutional.Conv2D object at 0x7f06c9fe33d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f06c9febb50>

In [ ]:
import numpy as np
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.utils.vis_utils import plot_model
def define_model(base_model, num_cat):
    inputs1 = Input(shape=(None, None, 3,))
    # Create the model
    model = Sequential()

    # Add the vgg convolutional base model
    model.add(base_model)

    # Add new layers
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_cat, activation='softmax'))

    # tie it together
    #model = Model(inputs=inputs1, outputs=output)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    # summarize model
    print(model.summary())
    plot_model(model, to_file='mode.png', show_shapes=True)
    return model

In [ ]:
model = define_model(base_model, 8)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
from collections import Counter

counter = Counter(train_generator.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : np.minimum(max_val/num_images,3) for class_id, num_images in counter.items()}
print(class_weights)

In [ ]:
from keras.callbacks import ModelCheckpoint
nb_train_samples = train_df.size
nb_validation_samples = valid_df.size
epochs=10
print(nb_train_samples, nb_validation_samples)
filepath="toy-model_1-epoch-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto',period=1)
callbacks_list = [checkpoint]




In [ ]:

history = model.fit(
        train_generator,
        steps_per_epoch=int(train_generator.n/batch_size),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=int(validation_generator.n/batch_size),
        callbacks=callbacks_list,
        class_weight=class_weights)

model.save('toy_model1.h5')

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


NameError: ignored

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory="data/test/",
    target_size=(img_h, img_w),
    color_mode="rgb",
    batch_size=64,
    class_mode=None,
    shuffle=False,
    seed=42
)

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
                                steps=STEP_SIZE_TEST,
                                verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:


filenames= [i.split('/')[1] for i in test_generator.filenames]
results=pd.DataFrame({"image":filenames,
                      "emotion":predictions})
results.to_csv("results.csv",index=False)
test_df.to_csv("ground_truth.csv")



In [ ]:
result_test_merged = pd.merge(results, test_df, left_on=['image'],
              right_on=['image'],
              how='inner')
compare=result_test_merged['emotion_x']==result_test_merged['emotion_y']
acc = sum(compare)/len(compare)
acc

### Convert model to TensorFlow Lite

In [ ]:
import tensorflow as tf
keras_model = tf.keras.models.load_model("toy_model1.h5")
converter =  (keras_model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)